In [ ]:
%pip install aeon

In [1]:
import numpy as np
import keras
from keras import layers
import pandas as pd
from sklearn.model_selection import train_test_split
import json
import os
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import sklearn
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from aeon.classification.hybrid import HIVECOTEV2

In [2]:
from sklearn.metrics import accuracy_score
from aeon.classification.hybrid import HIVECOTEV2

In [11]:
file_path = '../combined_all_interpol_synced_4sec_800samples_prot4.pkl'

dataFrame = pd.read_pickle(file_path)


print(dataFrame.columns)
y = np.array(dataFrame['uid'])
x = None
for col in dataFrame.columns:
    if 'sensordata' in col:
        if x is None:
            x = np.array(dataFrame[col].apply(lambda x: np.array(x)).to_list())
            #print(x_data.shape)
        else:
            x_data = np.array(dataFrame[col].apply(lambda x: np.array(x)).to_list())
            #print(x_data.shape)
            x = np.append(x, x_data , axis=2)
dur = np.array(dataFrame['duration'])
print("Dataset shapes: ")
print(x.shape)
print(dur.shape)
print(y.shape)


def split_features_and_adjust_labels(x, y, window_size=100, overlap=0.4):
    num_samples = x.shape[0]
    num_timesteps = x.shape[1]
    num_features = x.shape[2]

    step_size = int(window_size * (1 - overlap))
    num_segments = int((num_timesteps - window_size) / step_size) + 1

    split_x = np.zeros((num_samples * num_segments, window_size, num_features))
    adjusted_y = np.repeat(y, num_segments)

    segment_index = 0
    for sample_index in range(num_samples):
        for i in range(0, num_timesteps - window_size + 1, step_size):
          split_x[segment_index] = x[sample_index, i : i + window_size, :]
          segment_index += 1

    return split_x, adjusted_y, num_segments

def get_aligned_and_rest(split_x, adjusted_y,num_segments):
    x_aligned = []
    y_aligned = []
    x_other = []
    y_other = []

    for i in range(len(adjusted_y)):
        if i % num_segments == 0:
            x_aligned.append(split_x[i])
            y_aligned.append(adjusted_y[i])
        else:
            x_other.append(split_x[i])
            y_other.append(adjusted_y[i])
    return np.array(x_aligned), np.array(y_aligned), np.array(x_other), np.array(y_other)


split_x, adjusted_y,num_segments  = split_features_and_adjust_labels(x,y,window_size=100,overlap=0)
print(split_x.shape, adjusted_y.shape)
x_aligned, y_aligned, x_other, y_other = get_aligned_and_rest(split_x,adjusted_y, num_segments)


y_aligned = y_aligned.astype(str)
y_other = y_other.astype(str)
x = x_aligned
y = y_aligned


class_label_encoder = LabelEncoder()
scaler = StandardScaler()

y = class_label_encoder.fit_transform(y)
y_other = class_label_encoder.transform(y_other)
n_classes = class_label_encoder.classes_.shape[0]

print("train-test shapes: ")
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
x_train = scaler.fit_transform(x_train.reshape(-1, x_train.shape[-1])).reshape(x_train.shape)
x_test = scaler.transform(x_test.reshape(-1, x_test.shape[-1])).reshape(x_test.shape)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

x_other_scaled = scaler.transform(x_other.reshape(-1, x_other.shape[-1])).reshape(x_other.shape)


print(x_other_scaled.shape, y_other.shape)

Index(['uid', 'sensordata.ACC', 'sensordata.GYR', 'sensordata.GRV',
       'sensordata.MAG', 'sensordata.ROT', 'duration'],
      dtype='object', name='stype')
Dataset shapes: 
(332, 800, 17)
(332,)
(332,)
(2656, 100, 17) (2656,)
train-test shapes: 
(232, 100, 17) (100, 100, 17) (232,) (100,)
(232, 100, 17) (100, 100, 17) (232,) (100,)
(2324, 100, 17) (2324,)


In [4]:
with tf.device('/GPU:0'):
  # Fit HC2
  hc2 = HIVECOTEV2(verbose=1,n_jobs = -1, )
  hc2.fit(x_train, y_train)

  # Predict and print accuracy
  predictions = hc2.predict(x_test)
  print(accuracy_score(y_test, predictions))

STC  14:35:19 10/12/2024
STC weight = 0.6096260332179014
DrCIF  14:39:28 10/12/2024
DrCIF weight = 0.6713132812930787
Arsenal  14:39:33 10/12/2024
Arsenal weight = 0.5747163311417245
TDE  14:58:33 10/12/2024
TDE weight = 0.6096260332179014
0.84


In [12]:
with open('hc2_model.pkl', 'rb') as file:
    hc2 = pickle.load(file)

# Verify the loaded model
print(hc2)

HIVECOTEV2(n_jobs=-1, verbose=1)


In [13]:
pred = hc2.predict(x_test)

In [15]:
print(sklearn.metrics.classification_report(y_test, pred))
#print(sklearn.metrics.classification_report(y_other, predicted_other))

              precision    recall  f1-score   support

           0       0.79      0.94      0.86        16
           1       0.72      0.76      0.74        17
           2       0.73      0.65      0.69        17
           3       1.00      1.00      1.00         3
           4       0.90      0.75      0.82        12
           5       1.00      0.50      0.67         2
           6       1.00      0.90      0.95        10
           7       1.00      1.00      1.00         7
           8       0.70      1.00      0.82         7
           9       1.00      1.00      1.00         5
          10       1.00      0.75      0.86         4

    accuracy                           0.83       100
   macro avg       0.90      0.84      0.85       100
weighted avg       0.84      0.83      0.83       100



In [6]:
print(hc2.summary())

AttributeError: 'HIVECOTEV2' object has no attribute 'summary'

In [7]:
pickle.dump(hc2, open('hc2_model.pkl', 'wb'))

